In [52]:
library(devtools)
load_all()

ℹ Loading ghostbasil


In [53]:
generate.data <- function(n, p, seed, s)
{
    set.seed(seed)
    X <- matrix(rnorm(n * p), n, p)
    y <- X %*% rnorm(p) + rnorm(n)
    Sigma <- t(X) %*% X / n
    Sigma <- (1-s) * Sigma
    diag(Sigma) <- diag(Sigma) + s
    r <- t(X) %*% y / n
    list(Sigma=Sigma, r=r)
}

generate.data.block <- function(n, p, L, seed=0, s=0)
{
    data <- lapply(1:L, function(l) generate.data(n, p, seed + l - 1, s))
    mat.list <- lapply(1:L, function(l) data[[l]]$Sigma)
    r <- as.numeric(sapply(1:L, function(l) data[[l]]$r))
    A <- BlockMatrix(mat.list)
    A.dense <- A$to_dense()
    list(
        A=A,
        A.dense=A.dense,
        r=r
    )
}

In [54]:
n <- 10
p <- 1000
L <- 1
seed <- 39
s <- 0

data <- generate.data.block(n, p, L, seed, s)
A.bm <- data$A
A.dm <- data$A.dense
r <- data$r

In [55]:
alpha <- 1

In [56]:
out <- ghostbasil(A.dm, r, alpha=alpha, max.lambdas=100)

In [57]:
lmdas <- out$lmdas[1:100]
out <- ghostbasil(A.dm, r, alpha=alpha, user.lambdas=lmdas, lambdas.iter=1)

In [58]:
out.par <- ghostbasil(A.bm, r, alpha=alpha, n.threads=1, user.lambdas=lmdas, lambdas.iter=1)

In [59]:
# strong set
sort(c(out.par$checkpoint[[1]]$strong_set, out.par$checkpoint[[2]]$strong_set + p))
sort(out$checkpoint$strong_set)

# grad
(out$checkpoint$grad - c(out.par$checkpoint[[1]]$grad, out.par$checkpoint[[2]]$grad))

# rsq
out$checkpoint$rsq
out.par$checkpoint[[1]]$rsq + out.par$checkpoint[[2]]$rsq

out$rsqs[1:length(lmdas)]
out$diagnostic
out.par$diagnostic

ERROR: Error in out.par$checkpoint[[2]]: 첨자의 허용 범위를 벗어났습니다


In [61]:
(out$rsqs-out.par$rsqs)
sum(out$lmdas != out.par$lmdas)
sum(out$rsqs != out.par$rsqs)
sum(out$betas != out.par$betas)

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

[1] 0

[1] 0

[1] 0

In [ ]:
obj <- function(A, r, beta, lmda, penalty=rep(1, length(r)), alpha=1)
{
    objective_dense__(A, r, penalty, alpha, lmda, beta)
}

In [ ]:
objs <- sapply(1:length(out$lmdas), 
    function(i) {
        c(obj(A.dm, r, out$beta[, i], out$lmdas[i])$objective,
          obj(A.dm, r, out.par$beta[, i], out.par$lmdas[i])$objective)
    })
colors <- c('black', 'red')[(objs[1,] < objs[2,]) + 1]
#plot(objs[1,], objs[2,], col=colors)
plot((objs[1,]-objs[2,]) / abs(objs[1,]))
which(objs[1,]-objs[2,] == min(objs[1,]-objs[2,]))